In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,430 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-03 23:26:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-03 23:26:29 (6.46 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
# Remove duplicates
df = df.dropDuplicates()

In [13]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({
    "customer_id": "count"
    }).withColumnRenamed("count(customer_id)", "customer_count")
customers_df = customers_df.dropDuplicates(["customer_id"])
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51163715|             6|
|   12134452|             1|
|   52953209|           110|
|   50926345|             1|
|   17235720|             2|
|   50755203|            26|
|   15996792|             1|
|   52222838|             3|
|   20469836|            41|
|   25459059|            34|
|   50093726|           151|
|   12826625|             1|
|   52938787|             2|
|   52507682|             1|
|   50994048|            27|
|   52922003|             6|
|   48313694|             1|
|   50978179|             1|
|   35129585|             1|
|   51893641|             5|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates(
df.count()
products_df = df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001714422|'C' Is for Clown ...|
|0001837192|Pieter Brueghel's...|
|0002005565|             Thieves|
|0002151731|The Greatest Bene...|
|0002216973|     Red Adam's Lady|
|0002217236|       Running Blind|
|000255111X|The Adventures of...|
|0004708237|Collins Gem Shubu...|
|0006544150|           In Xanadu|
|000717201X|The Lord of the R...|
|0007640617|Energize Your Lif...|
|0020710518|Mud Soldiers: Lif...|
|002075910X|Group Marriage;: ...|
|0020960107|The Only Way to C...|
|0023605103|Parables of Jesus...|
|0025239716|The Baseball Chro...|
|0025703811|Life on the Rim: ...|
|0027182223|The CHIMES OF ALY...|
|0027701808|             Dogsong|
|0028610881|Complete Idiot's ...|
+----------+--------------------+
only showing top 20 rows



In [15]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# reviews_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(["review_id", "customer_id", "product_id","product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.dropDuplicates(["review_id"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1009IZWGTHPVN|   51002723|9057551195|      84154346| 2001-01-29|
|R100AVKGGUXS92|   51164957|0027041603|     320106897| 2001-09-06|
|R100V1EXE75152|   52536381|0195167716|     960527707| 2004-10-12|
|R101NKOTH7FT5Q|   52654147|0130868264|     149426396| 2001-01-15|
|R101UIKYRUTEC4|   50304821|0849938341|     127562478| 2000-03-26|
|R101VRLY2NU91H|   15875814|0312050631|     800037606| 2004-12-11|
|R1020FRYZ1LSQA|   49480574|0761513892|     358493031| 2000-05-25|
|R1022VT6V7IC0Y|   32106090|0486269183|     371594208| 2003-02-05|
|R102ASWN15DHH6|   51781902|0807847615|     517030876| 1999-05-22|
|R102B6IEA36O7Y|   50592769|0525944877|     826374705| 2000-03-06|
|R102RZPC872T2K|   49546072|014019536X|     990968702| 2000-05-25|
|R1039WSWSNLJ1K|   13871683|0974435406|     425668955| 2005-04

In [16]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine","verified_purchase"])
vine_df = vine_df.dropDuplicates(["review_id"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1009IZWGTHPVN|          5|            2|          5|   N|                N|
|R100AVKGGUXS92|          3|            0|          2|   N|                N|
|R100V1EXE75152|          3|            9|         16|   N|                N|
|R101NKOTH7FT5Q|          4|            4|          4|   N|                N|
|R101UIKYRUTEC4|          5|            3|          3|   N|                N|
|R101VRLY2NU91H|          5|           12|         13|   N|                N|
|R1020FRYZ1LSQA|          1|            2|          2|   N|                N|
|R1022VT6V7IC0Y|          1|            0|         22|   N|                N|
|R102ASWN15DHH6|          5|            8|          8|   N|                Y|
|R102B6IEA36O7Y|          3|            1|          2|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonreviewchallengewj.cl4kfbxncu9y.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)